# audiolazy

[back to overview page](index.ipynb)

http://pythonhosted.org/audiolazy/getting_started.html

In [ ]:
import audiolazy as lz
from audiolazy import z
%matplotlib inline

In [ ]:
a = lz.Stream(2, -2, -1)
a

In [ ]:
b = lz.Stream(3, 7, 5, 4)

In [ ]:
c = a + b
c

In [ ]:
c.take(15)

In [ ]:
a = lz.Stream([1, 2, 3, 2, 1])  # finite

In [ ]:
b = lz.Stream(3, 7, 5, 4)  # periodic

In [ ]:
c = a + b

In [ ]:
list(c)

In [ ]:
filt = 1 - z ** -1
filt

In [ ]:
type(filt)

In [ ]:
filt.is_causal(), filt.is_lti()

In [ ]:
data = filt([.1, .2, .4, .3, .2, -.1, -.3, -.2])
data

In [ ]:
data *= 10

In [ ]:
list(data)

In [ ]:
(1 + z ** -2).plot()

In [ ]:
filt.plot()

In [ ]:
filt.zplot()

In [ ]:
filt1 = lz.CascadeFilter(0.2 - z ** -3)  # 3 zeros
filt1

In [ ]:
filt2 = lz.CascadeFilter(1 / (1 -.8 * z ** -1 + .6 * z ** -2))  # 2 poles
filt2

In [ ]:
# Here __add__ concatenates and __mul__ by an integer make reference copies
filt = (filt1 * 5 + filt2 * 10) # 15 zeros and 20 poles
filt

In [ ]:
filt.zplot()

In [ ]:
lz.lpc([1, -2, 3, -4, -3, 2, -3, 2, 1], order=3).plot()

In [ ]:
data = lz.Stream(-1., 0., 1., 0.)  # periodic
blk = data.take(200)

In [ ]:
analysis_filt = lz.lpc.covar(blk, 4)
analysis_filt

In [ ]:
residual = analysis_filt(blk)
residual

In [ ]:
residual.take(10)

In [ ]:
synth_filt = 1 / analysis_filt
synth_filt(residual).take(10)

In [ ]:
amplified_blk = list(lz.Stream(blk) * -200)  # for alignment w/ DFT

In [ ]:
synth_filt.plot(blk=amplified_blk)

In [ ]:
rate = 44100  # sampling rate, in samples/second
s, Hz = lz.sHz(rate) # seconds and Hertz
ms = 1e-3 * s

In [ ]:
s, Hz

In [ ]:
note1 = lz.karplus_strong(440 * Hz)  # pluck "digitar" synth
note1

In [ ]:
note2 = lz.zeros(300 * ms).append(lz.karplus_strong(880 * Hz))
note2

In [ ]:
notes = (note1 + note2) * .5

In [ ]:
sound = notes.take(int(2 * s))  # 2 seconds of a Karplus-Strong note

In [ ]:
with lz.AudioIO(True) as player:  # True means "wait for all sounds to stop"
    player.play(sound, rate=rate)

In [ ]:
import sounddevice as sd

In [ ]:
sd.play(sound, rate)

http://pythonhosted.org/audiolazy/audiolazy.html

In [ ]:
delay_a4 = lz.freq_to_lag(440 * Hz)
delay_a4

In [ ]:
filt = lz.ParallelFilter(lz.comb.tau(delay_a4, 20 * s), lz.resonator(440 * Hz, bandwidth=100 * Hz))
filt

In [ ]:
len(filt)

In [ ]:
lz.comb.tau(3)

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span resource="[_:publisher]" rel="dct:publisher">
    <span property="dct:title">Matthias Geier</span></span>
  has waived all copyright and related or neighboring rights to
  this work.
</p>